<a href="https://colab.research.google.com/github/swafe/Programms/blob/master/sort_sample_and_convert_polygon_json_to_png.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script creates 
1. a new sorted json file, where the regions are sorted from top to bottom
and 
2. cuts the detected regions from the original image and saves as e new cutted image. The new filename consists of the filename( image number, 'BKF', sample mumber/location) and sample id

In [89]:
import shutil
import os

import numpy as np
import json 

from PIL import Image, ImageDraw


In [ ]:
#falls git existiert löschen
path_data = '/content/data'
if os.path.exists(path_data):
  print('g')
  shutil.rmtree(path_data)
  #os.remove(path_data)
!git clone https://github.com/swafe/data.git

In [108]:
# some Methods

def get_access_to_json(image_path):
  image_name = os.path.splitext(os.path.basename(image_path))[0]
  filename = os.path.basename(image_path)
  img_size =  os.path.getsize(image_path)
  filename_and_size = filename + str(img_size)
  return filename_and_size, filename

# Iterating through the json 
# list 
def get_points_label_from_json_as_array(data, image_path):
  detected = []
  sample_id = []
  filename_and_size, filename = get_access_to_json(image_path)
  for i in data[filename_and_size]['regions']: #[filename_and_size]: 
    # print(i)
    points_x = i['shape_attributes']['all_points_x']
    points_y = i['shape_attributes']['all_points_y']
    sample_id_i = i["region_attributes"] ["sample"]
    # # für ImageDraw.Draw tuple erforderlich
    points =[]
    for i in range(0,len(points_x)):
      points_i = (points_x[i],points_y[i])
      points.append(points_i)

    detected.append(points)
    sample_id.append(sample_id_i)
  # print(np.shape(detected))
  return detected, filename_and_size, filename, sample_id


In [92]:
# Python program to read json file 
# change here !!!!!!
path_test_dir = '/content/data/vgg_annotation/img_horizontal/project_8205_1/prepare_test_for_new_train_02/test'
json_name = 'via_project_25Jun2020_22h5m_json.json'
json_file_dir = os.path.join(path_test_dir, json_name) 

#get all images in path_test_dir
image_paths = []
for filename in sorted(os.listdir(path_test_dir), key=str.lower):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
      image_paths.append(os.path.join(path_test_dir, filename))
# Opening JSON file 
f = open(json_file_dir) 
# returns JSON object as a dictionary 
data = json.load(f) 
# Closing file 
f.close() 

In [ ]:
#sorting regions in json from upside to bottom
# https://stackoverflow.com/questions/26924812/python-sort-list-of-json-by-value
# Sorting objects doesn't make sense since object keys have no positional value. 

# // First create the array of keys/net_total so that we can sort it:
#store
data_sorted = data
# i= 1
# image_path = image_paths[i]
for path_i in image_paths:
#path_i = image_paths[2]


  ## get points_x_all, points_y_all, sample_id
  sample_id = []
  points_x_all = []
  points_y_all = []
  regions_all=[]
  # labels = []
  filename_and_size, filename = get_access_to_json(path_i)

  for i in data[filename_and_size]['regions']: #[filename_and_size]: 
      regions_all.append(i)
  #    print(len(regions_all))
  for i_regions in regions_all:
      points_x =  i_regions['shape_attributes']['all_points_x']
      points_y =  i_regions['shape_attributes']['all_points_y']
      points_x_all.append(points_x)
      points_y_all.append(points_y)

  ##
  # // Now sort it:
  print(points_y_all)
  points_y_all_first = []
  for i_y in points_y_all:
    points_y_all_first_i = i_y[0]
    points_y_all_first.append(points_y_all_first_i)
  print(points_y_all_first)
  sort_index_min_to_max = np.argsort(points_y_all_first)#[::-1]
  print(sort_index_min_to_max)
  # create regions with new values
  regions_mask_rois_adapted = []
  i_sample = 0
  for i_sort in sort_index_min_to_max:
    regions_i_adapted = {"shape_attributes": {
                                  "name": "polygon",
                                  "all_points_x": points_x_all[i_sort], 
                                  "all_points_y": points_y_all[i_sort] },
                                  "region_attributes": { "sample": int(i_sample)}}        
    regions_mask_rois_adapted.append(regions_i_adapted)
    i_sample = i_sample + 1
  # annotation_json_via_format_rois_mask_adapted_i = {
  #                   "filename": filename, 
  #                   "size": img_size, 
  #                   "regions": regions_mask_rois_adapted,
  #                   "file_attributes": {}
  #                   }
  # update json with new regions
  # annotation_json_via_format.update({filename_and_size : annotation_json_via_format_rois_mask_adapted_i})
  data_sorted.update({filename_and_size:{"regions":regions_mask_rois_adapted}})
# save annotation as json
if True:
  with open('result_masks_data_sorted.json', 'w') as fp:
      json.dump(data_sorted, fp)

In [ ]:
# cutting detected images 
# https://stackoverflow.com/questions/22588074/polygon-crop-clip-using-python-pil

# read image as RGB and add alpha (transparency)
for path_i in image_paths:
  im = Image.open(path_i).convert("RGBA")

  # convert to numpy (for convenience)
  imArray = np.asarray(im)
  # create mask
  detected, filename_and_size, filename, sample_id = get_points_label_from_json_as_array(data_sorted, path_i)

  #polygon = [(444,203),(623,243),(691,177),(581,26),(482,42)]
  for i in range(np.shape(detected)[0]):
    sample_id_i = sample_id[i]
    polygon = detected[i]
    #print(polygon)
    maskIm = Image.new('L', (imArray.shape[1], imArray.shape[0]), 0)
    ImageDraw.Draw(maskIm).polygon(polygon, outline=1, fill=1)
    mask = np.array(maskIm)

    # assemble new image (uint8: 0-255)
    newImArray = np.empty(imArray.shape,dtype='uint8')

    # colors (three first columns, RGB)
    newImArray[:,:,:3] = imArray[:,:,:3]

    # transparency (4th column)
    newImArray[:,:,3] = mask*255

    # back to Image from numpy
    newIm = Image.fromarray(newImArray, "RGBA")
    # save snipped images in folder strukture, where snipped images are stored in folder named as origin image
    if False: # True False
      path = '/content/snipped_img/%s'%(filename)
      print(path)
      if not os.path.exists(path):
        os.makedirs(path)
      newIm.save("%s/%s.png"%(path,i))

    # save snipped image with name of originimage plus index
    if True: # True False
      path = '/content/snipped_img'
      if not os.path.exists(path):
        os.makedirs(path)
      newIm.save("%s/%s%s.png"%(path,filename,sample_id_i))
      print(filename,sample_id_i)

old code following 

In [36]:
# delete folder 
# path_data = '/content/snipped_img'
# if os.path.exists(path_data):
#   print(path_data)
#   shutil.rmtree(path_data)

In [ ]:
!zip -r /content/file.zip /content/snipped_img

from google.colab import files
files.download("/content/file.zip")

In [ ]:
# Python program to read/show images using matplotlib 

# importing matplotlib modules 
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
# Read Images 
# img = mpimg.imread('/content/Img_0107BKF 18_11.jpg/0.png') 
img = plt.imread('/content/Img_0107BKF 18_11.jpg/0.png') 
# Output Images 
plt.imshow(img) 
